In [1]:
pwd

'd:\\OneDrive\\바탕 화면\\colle\\Git\\concept_centric_transformers'

In [ ]:
!pip install -r requirements.txt

### embryo 파인튜닝

In [ ]:
!!python embryo_unsup.py \
    --backbone swin_tiny \
    --slot_type qsa \
    --lr 2e-5 \
    --warmup_epochs 5 \
    --freeze_stages 1 \
    --lambda_sparse 2.0 \
    --lambda_div 2.0 \
    --epochs 50 \
    --weight_decay 0.05 \
    --root ./embryo

### 시각화

In [2]:
!python vis_embryo.py \
  --ckpt_path checkpoints/lightning_logs/version_6/checkpoints/epoch=39-step=1440.ckpt \
  --top_k 20 --thres 0.5 --n_unsup 4

🔹 checkpoint 로드 중 …
🔹 test set forward (예측) …
after squeeze : torch.Size([82, 4, 196])
🔹 시각화 저장 …
✅ slot0: 30장 저장 완료
✅ slot1: 30장 저장 완료
✅ slot2: 30장 저장 완료
✅ slot3: 30장 저장 완료
🎉 완료!


INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations
c:\Users\nunaj\anaconda3\envs\cct_py310\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
c:\Users\nunaj\anaconda3\envs\cct_py310\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:92: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality m

In [22]:
!python channel_check.py

총 이미지  : 812
RGB(3채널) : 641  → rgb_only 에 복사 완료


### 성능평가

In [9]:
!python eval_blasto_cct.py \
  --checkpoint checkpoints/lightning_logs/version_14/checkpoints/epoch=10-step=396.ckpt \
  --model_entry "ctc.cct_builder:build_model" \
  --img_size 224 --batch_size 16 --device cuda \
  --auc --saliency --area_prec \
  --dataset_root ./embryo \
  --freeze_stages 1 --lambda_div 2.0 --lambda_sparse 2.0 \
  --warmup_epochs 5 --lr 2e-5 \
  --loss_status 1

[Info] froze forse 1 stage
[Info] generating explanation images …
AUC: 0.594329
AUC: 0.568217
AUC: 0.577837
AUC: 0.555309
AUC: 0.582565
AUC: 0.564393
AUC: 0.589004
AUC: 0.570822
AUC: 0.571447
AUC: 0.569153
AUC: 0.531757
AUC: 0.518246
AUC: 0.588912
AUC: 0.566986
AUC: 0.583269
AUC: 0.555273
AUC: 0.598812
AUC: 0.575993
AUC: 0.521818
AUC: 0.532211
AUC: 0.542247
AUC: 0.542420
IAUC=0.5711 | DAUC=0.5563


INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations
c:\Users\nunaj\anaconda3\envs\cct_py310\lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\nunaj\anaconda3\envs\cct_py310\lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\nunaj\anaconda3\envs\cct_py310\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/

In [ ]:
!python eval_blasto_cct.py \
  --checkpoint checkpoints/lightning_logs/version_6/checkpoints/epoch=39-step=1440.ckpt \
  --model_entry "models.cct_builder:build_model" \
  --img_size 224 --batch_size 16 --device cuda \
  --auc --saliency \
  --dataset_root ./embryo \
  --loss_status -1

### confusion_matrix

!python confusion_matrix_metrics_cct.py \
  --checkpoint checkpoints/lightning_logs/version_14/checkpoints/epoch=10-step=396.ckpt \
  --model_entry ctc.cct_builder:build_model \
  --dataset_root ./embryo --split test \
  --img_size 224 --batch_size 16 --device cuda \
  --backbone swin_tiny --slot_type qsa --num_classes 2 \
  --n_unsup 4 --n_concepts 0 --n_spatial 4 \
  --pos_label 1